In [96]:
import re
import urllib.request
from bs4 import BeautifulSoup
import json
from itertools import permutations
import numpy as np
import pandas as pd

#Read Data in From DraftKings
url = "https://sportsbook.draftkings.com/api/odds/v1/leagues/3/offers/gamelines.json"
request = urllib.request.Request(url)
page = urllib.request.urlopen(request).read()
b = json.loads(page)
b['errorStatus']


{'code': 'BET419',
 'developerMessage': 'No Gamelines Events Found for League 3'}

In [97]:
url = "https://sportsbook.draftkings.com/api/odds/v1/leagues/"
request = urllib.request.Request(url)

page = urllib.request.urlopen(request).read()
b = json.loads(page)

league_df = pd.json_normalize(b, record_path =['leagues'])
print(league_df.head())

                                  name  leagueId    sportName  sportId
0              Malta - Premier League   88674310       Soccer     5312
1  ITF Doubles - Frankfurt am Main (M)  88674346       Tennis     5321
2   NASCAR Xfinity Series Championship  88674348  Motorsports     5325
3         ATP Qualifiers - Cincinnati   88674362       Tennis     5321
4                     ATP - Cincinnati  88674363       Tennis     5321


In [98]:
#Read Data in From DraftKings
url = "https://sportsbook.draftkings.com/api/odds/v1/leagues/3/offers/gamelines.json"
request = urllib.request.Request(url)
page = urllib.request.urlopen(request).read()
b = json.loads(page)
b['errorStatus']
'errorStatus' in b.keys()

True

In [215]:
ct = 0
print(len(league_df[league_df['sportName']=='Soccer']))

for i in [88670607]:# league_df.loc[league_df['sportName']=='Soccer', 'leagueId']:
    url = f"https://sportsbook.draftkings.com/api/odds/v1/leagues/{i}/offers/gamelines.json"
    print(i)
    request = urllib.request.Request(url)
    page = urllib.request.urlopen(request).read()
    b = json.loads(page)
    if 'errorStatus' in b.keys():
        print(ct, i)
        ct += 1
#page = urllib.request.urlopen(request).read()
b = json.loads(page)
b.keys()

236
88670607


dict_keys(['events'])

In [216]:

b0 = pd.json_normalize(b, record_path =['events'])

works_data = json_normalize(data=b['events'], record_path='works', 
                            meta=['id', 'orchestra','programID', 'season'])

In [211]:
soloist_data = pd.json_normalize(data=b0['offers'], record_path=['name', 'soloists'], 
                              meta=['id'])

,id,name,homeTeamName,awayTeamName,startDate,offers
0,180172178,Chelsea vs Tottenham,Chelsea,Tottenham,2022-01-23T16:30:00.0000000Z,"[{'id': '2-1_62514862', 'label': 'Moneyline', ..."
1,180186870,Tottenham vs Southampton,Tottenham,Southampton,2022-02-09T19:45:00.0000000Z,"[{'id': '2-3_62741786', 'label': 'Total', 'out..."
2,180186876,Norwich vs Crystal Palace,Norwich,Crystal Palace,2022-02-09T19:45:00.0000000Z,"[{'id': '2-3_62741785', 'label': 'Total', 'out..."
3,180186868,Wolves vs Arsenal,Wolves,Arsenal,2022-02-10T19:45:00.0000000Z,"[{'id': '2-2_62741923', 'label': 'Asian Handic..."
4,180186875,Burnley vs Man Utd,Burnley,Man Utd,2022-02-08T20:00:00.0000000Z,"[{'id': '2-105143599', 'label': 'Tie No Bet', ..."
5,180186871,Newcastle vs Everton,Newcastle,Everton,2022-02-08T19:45:00.0000000Z,"[{'id': '2-2_62741827', 'label': 'Asian Handic..."
6,180172174,Leicester vs Brighton,Leicester,Brighton,2022-01-23T14:00:00.0000000Z,"[{'id': '2-3_62514992', 'label': 'Total Up to ..."
7,180172176,Crystal Palace vs Liverpool,Crystal Palace,Liverpool,2022-01-23T14:00:00.0000000Z,"[{'id': '2-2_62514979', 'label': 'Asian Handic..."
8,180192509,Burnley vs Watford,Burnley,Watford,2022-02-05T18:00:00.0000000Z,"[{'id': '2-2_62828092', 'label': 'Asian Handic..."
9,180186877,West Ham vs Watford,West Ham,Watford,2022-02-08T19:45:00.0000000Z,"[{'id': '2-3_62741814', 'label': 'Total Up to ..."


In [174]:
b

{'events': [{'id': 180172178,
   'name': 'Chelsea vs Tottenham',
   'homeTeamName': 'Chelsea',
   'awayTeamName': 'Tottenham',
   'startDate': '2022-01-23T16:30:00.0000000Z',
   'offers': [{'id': '2-1_62514862',
     'label': 'Moneyline',
     'outcomes': [{'id': '2-0ML62514862_1',
       'label': 'Chelsea',
       'oddsAmerican': '-145',
       'oddsDecimal': 1.68965518,
       'oddsFractional': '20/29',
       'participant': 'Chelsea'},
      {'id': '2-0ML62514862_2',
       'label': 'Draw',
       'oddsAmerican': '+285',
       'oddsDecimal': 3.85,
       'oddsFractional': '57/20',
       'participant': 'Draw'},
      {'id': '2-0ML62514862_3',
       'label': 'Tottenham',
       'oddsAmerican': '+425',
       'oddsDecimal': 5.25,
       'oddsFractional': '17/4',
       'participant': 'Tottenham'}]},
    {'id': '2-2_62515000',
     'label': 'Spread',
     'outcomes': [{'id': '2-0HC62515000N250_1',
       'label': 'Chelsea',
       'line': '-2.5',
       'oddsAmerican': '+600',
      

In [164]:
ay = []
for i in b0.loc[0].offers:
    print(i['label'])
    for ii in i['outcomes']:
        ay.append(ii)
    #ay.append(pd.json_normalize(i, record_path =['outcomes']))
all_whatever = pd.DataFrame(ay)
#pd.concat(ay)

Moneyline
Asian Handicap Up to 30th Minute
Asian Handicap Up to 70th Minute
Total Up to 30th Minute
Tie No Bet
Total
Total Up to 70th Minute
Total Goals
Spread
Asian Handicap


In [161]:

b0.loc[0].offers

[{'id': '2-1_62741782',
  'label': 'Moneyline',
  'outcomes': [{'id': '2-0ML62741782_1',
    'label': 'West Ham',
    'oddsAmerican': '-235',
    'oddsDecimal': 1.42553192,
    'oddsFractional': '20/47',
    'participant': 'West Ham'},
   {'id': '2-0ML62741782_2',
    'label': 'Draw',
    'oddsAmerican': '+360',
    'oddsDecimal': 4.6,
    'oddsFractional': '18/5',
    'participant': 'Draw'},
   {'id': '2-0ML62741782_3',
    'label': 'Watford',
    'oddsAmerican': '+600',
    'oddsDecimal': 7.0,
    'oddsFractional': '6/1',
    'participant': 'Watford'}]},
 {'id': '2-2_62741814',
  'label': 'Asian Handicap Up to 30th Minute',
  'outcomes': [{'id': '2-0HC62741814N25_1',
    'label': 'West Ham',
    'line': '-0.25',
    'oddsAmerican': '-120',
    'oddsDecimal': 1.83333334,
    'oddsFractional': '5/6'},
   {'id': '2-0HC62741814P25_3',
    'label': 'Watford',
    'line': '+0.25',
    'oddsAmerican': '-125',
    'oddsDecimal': 1.8,
    'oddsFractional': '4/5'}],
  'main': True},
 {'id': '2

In [90]:
b1 = pd.json_normalize(league_df['offers'].iloc[0], record_path =['outcomes'])

In [ ]:
# how much you wiin - wagers
# Group on Line 
# Bet 100 on Plus 100
2-0ML62741785_1 - win
2-0ML62741785_2 draw (not wanted)
2-0ML62741785_3 - loss


In [85]:
b1 = pd.json_normalize(league_df['offers'].iloc[0])

In [92]:
b0

,id,name,homeTeamName,awayTeamName,startDate,offers
0,180186876,Norwich vs Crystal Palace,Norwich,Crystal Palace,2022-02-09T19:45:00.0000000Z,"[{'id': '2-105143667', 'label': 'Tie No Bet', ..."
1,180186871,Newcastle vs Everton,Newcastle,Everton,2022-02-08T19:45:00.0000000Z,"[{'id': '2-1_62741783', 'label': 'Moneyline', ..."
2,180172178,Chelsea vs Tottenham,Chelsea,Tottenham,2022-01-23T16:30:00.0000000Z,"[{'id': '2-3_62514862', 'label': 'Total', 'out..."
3,180186877,West Ham vs Watford,West Ham,Watford,2022-02-08T19:45:00.0000000Z,"[{'id': '2-2_62741810', 'label': 'Spread', 'ou..."
4,180186873,Aston Villa vs Leeds,Aston Villa,Leeds,2022-02-09T20:00:00.0000000Z,"[{'id': '2-3_62741891', 'label': 'Total Goals'..."
5,180186868,Wolves vs Arsenal,Wolves,Arsenal,2022-02-10T19:45:00.0000000Z,"[{'id': '2-105144007', 'label': 'Tie No Bet', ..."
6,180192509,Burnley vs Watford,Burnley,Watford,2022-02-05T18:00:00.0000000Z,"[{'id': '2-3_62828220', 'label': 'Total Up to ..."
7,180172177,Arsenal vs Burnley,Arsenal,Burnley,2022-01-23T14:00:00.0000000Z,"[{'id': '2-2_62514913', 'label': 'Asian Handic..."
8,180172176,Crystal Palace vs Liverpool,Crystal Palace,Liverpool,2022-01-23T14:00:00.0000000Z,"[{'id': '2-2_62514860', 'label': 'Asian Handic..."
9,180186875,Burnley vs Man Utd,Burnley,Man Utd,2022-02-08T20:00:00.0000000Z,"[{'id': '2-2_62741851', 'label': 'Asian Handic..."
